
## Machine Learning Model
- neural ODE
- residual neural net -> SkipConnection
- parameter optimization
- stable vs less stable attractors

We now come to the question of approximating this system based on trajectory data. The method of Chen et. al. suggests considering a recurrent neural network as a "sequence of transformations to a hidden state $\mathbf{h}_t$" [SOURCE]:
$$
\mathbf{h}_{t+1} = \mathbf{h}_t + f(\mathbf{h}_t,\, \theta)
$$
where $t = 0, \ldots, T,\,\ \mathbf{h}_t \in \mathbb{R}^d$, and $f$ is a single layer of a neural network. They then extend this to an ODE propagating the state of a hidden layer
$$
\frac{d \mathbf{h} (t)}{d t} = f(\mathbf{h}(t),\, t,\, \theta)
$$
where $t \in [0, T]$, and $f$ is now an entire neural network. 

When considering a chaotic dynamical system, the "chaotic element" arises due to a nonlinearity in the system. This fact is particularly present in Chua's circuit, where the model is _almost_ entirely linear, save for one nonlinearity in the first component. Hence it is reasonable to desire that the neural network should contain a "linear" component, as well as a nonlinear one. Certainly there are many ways to achieve such a network; two possibilities are presented below. 

The first idea which comes to mind is to separate linear and nonlinear components explicitly in the network. More precisely, the neural network should model a function $nn(x) = W x + g(x)$ where $W \in \mathbb{R}^{d \times d}$ and $g$ can be considered as an explicit nonlinearity. This can be implemented using Flux.jl's built in layers. 

In [1]:
using Flux

In [2]:
N_weights = 15

W = Dense(3 => 3)   # dimension = 3

g = Chain(
    Dense(3 => N_weights),
    Dense(N_weights => N_weights, swish),
    Dense(N_weights => 3)
)

nn = Parallel(+, W, g)

Parallel(
  +,
  Dense(3 => 3),                        # 12 parameters
  Chain(
    Dense(3 => 15),                     # 60 parameters
    Dense(15 => 15, swish),             # 240 parameters
    Dense(15 => 3),                     # 48 parameters
  ),
)                   # Total: 8 arrays, 360 parameters, 2.000 KiB.

Another option would be to build in the linearity _implicitly_. One could use a more "typical" neural network with linear input and/or output layers, and use residual network layers to compute a nonlinearity. This allows the network model to "decide" for itself whether the linear component is strictly necessary. A concrete implementation would look like:

In [3]:
nn = Chain(
    Dense(3 => N_weights),

    SkipConnection(Dense(N_weights => N_weights, swish), +),
    SkipConnection(Dense(N_weights => N_weights, swish), +),
    SkipConnection(Dense(N_weights => N_weights, swish), +),

    Dense(N_weights => 3)
)

Chain(
  Dense(3 => 15),                       # 60 parameters
  SkipConnection(
    Dense(15 => 15, swish),             # 240 parameters
    +,
  ),
  SkipConnection(
    Dense(15 => 15, swish),             # 240 parameters
    +,
  ),
  SkipConnection(
    Dense(15 => 15, swish),             # 240 parameters
    +,
  ),
  Dense(15 => 3),                       # 48 parameters
)                   # Total: 10 arrays, 828 parameters, 3.906 KiB.

This model presents itself as a more "typical" residual neural network as described in literature. However, one observes by considering the network graph that this model also permits a linear and nonlinear component. Further, this structure benefits from the design of a residual neural network: the "vanishing gradient" problem is reduced [SOURCE], and extra layers can be added with reduced fear of overfitting since the model can simply "choose" to ignore unnecessary layers. 

During development of the project, both methods were tested and the implicit form performed better. However, the difference was not large. For the remainder of this report, the implicit method will be used. 

In [4]:
using WGLMakie

In [22]:
# for plotting training results
function plot_nde(sol, model, train; ndata=300)
    t = sol.t[1:ndata]
    pred = Array(model((t,train[1][2])))
    tr = Array(sol)
    fig, ax, ms = lines(t, pred[1, 1:ndata], label="Neural ODE dim 1")
    lines!(ax, t, pred[2, 1:ndata], label="Neural ODE dim 2")
    lines!(ax, t, pred[3, 1:ndata], label="Neural ODE dim 3")
    lines!(ax, t, tr[1, 1:ndata], label="Training Data dim 1")
    lines!(ax, t, tr[2, 1:ndata], label="Training Data dim 2")
    lines!(ax, t, tr[3, 1:ndata], label="Training Data dim 3")
    Legend(fig[1,2], ax)
    fig, ax
end

plot_nde (generic function with 1 method)

In [6]:
using StaticArrays, Statistics
using OrdinaryDiffEq, SciMLSensitivity#, CUDA
using NODEData, ChaoticNDETools

In [7]:
# Chua's circuit
function v(u, p, t)
    x, y, z = u
    a, b, m0, m1 = p
    SA{Float32}[ a*(y-m0*x-m1/3.0*x^3), x-y+z, -b*y ]
end

# parameters
p_ode = SA{Float32}[ 18.0, 33.0, -0.2, 0.01 ]
a, b, m0, m1 = p_ode

v(u) = v(u, p_ode, 0f0)

# equilibrium
x₊ = SA{Float32}[ sqrt(-3*m0/m1), 0, -sqrt(-3*m0/m1) ]
x₋ = -x₊

# integration time
t0, t1 = 0f0, 40f0
tspan = (t0, t1)
dt = 1f-2;

The data used will be the trajectories from the previous section. These are split into minibatches to both reduce the chance of the training model diverging, as well as to reduce condition number of the gadients for optimization. 

In [8]:
x0 = SA{Float32}[2, 1.5, 6]
prob = ODEProblem(v, x0, (t0, t1), p_ode)
sol = solve(prob, RK4(), saveat=dt, sensealg=InterpolatingAdjoint());

In [9]:
train, valid = NODEDataloader(sol, 8; dt=dt, valid_set=0.8, GPU=false#=true=#)
train

NODEData{Matrix{Float32},Int64} with 793 batches with length 8

The parameters of the model are extracted and flattened to a vector $p$ so that the gradient of the loss w.r.t. $p$ can be directly calculated. 

In [10]:
p, re_nn = Flux.destructure(nn)
#p = p |> gpu
neural_ode(u, p, t) = re_nn(p)(u)
neural_ode(u) = neural_ode(u, p, 0f0)

neural_ode_prob = ODEProblem(neural_ode, #=CuArray(x0)=#x0, tspan, p)
model = ChaoticNDE(neural_ode_prob, alg=RK4(), gpu=false#=true=#, sensealg=InterpolatingAdjoint())

ChaoticNDE{Vector{Float32}, ODEProblem{SVector{3, Float32}, Tuple{Float32, Float32}, false, Vector{Float32}, ODEFunction{false, SciMLBase.AutoSpecialize, typeof(neural_ode), LinearAlgebra.UniformScaling{Bool}, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, typeof(SciMLBase.DEFAULT_OBSERVED), Nothing, Nothing}, Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}, SciMLBase.StandardODEProblem}, RK4{typeof(OrdinaryDiffEq.trivial_limiter!), typeof(OrdinaryDiffEq.trivial_limiter!), Static.False}, Base.Pairs{Symbol, InterpolatingAdjoint{0, true, Val{:central}, Nothing}, Tuple{Symbol}, NamedTuple{(:sensealg,), Tuple{InterpolatingAdjoint{0, true, Val{:central}, Nothing}}}}, ChaoticNDETools.CPUDevice}(Float32[0.12409509, -0.11041498, 0.5740997, -0.3917037, -0.009971991, -0.37850785, 0.44426835, -0.037722163, -0.44841647, 0.12997003  …  0.16591525, -0.07331287, 0.4146641, -0.3196385, -0.2472412, -0.52597183, 0.042896

In [11]:
model(valid[1])

┌ Warning: Layer with Float32 parameters got Float64 input.
│   The input will be converted, but any earlier layers may be very slow.
│   layer = Dense(3 => 15)      # 60 parameters
│   summary(x) = "3-element Vector{Float64}"
└ @ Flux C:\Users\april\.julia\packages\Flux\Nzh8J\src\layers\stateless.jl:50


3×8 Matrix{Float32}:
  1.96761    1.96648    1.96541   …   1.96245    1.96155    1.96068
 -0.419014  -0.418694  -0.418249     -0.416171  -0.415236  -0.414182
  3.42939    3.41059    3.39192       3.33674    3.31863    3.30066

The final consideration required before training can be done is the loss function. The most naive loss function may be derived from the shooting method for boundary value problems. One integrates the model for some fixed time $T$, and compute the difference (in norm) of the model trajectory to the true trajectory data. This technique is extended analogoously to the method of _multiple_ shooting, where the multiple small consecutive trajectories are compared. The resulting differences can be added together to obtain a scalar valued loss function, equivalent (up to a scaling factor) to the mean squared error. 
$$
L(\mathbf{x}, \mathbf{\hat{x}}) = \| \mathbf{x} - \mathbf{\hat{x}} \| ^2 = \sum_{i=1}^d (\mathbf{x}_i - \mathbf{\hat{x}}_i) ^2
$$
Whle the mean squared error works quite well, a potential downfall can occur, particularly in periodic systems. In each small trajectory, the errors of the model will compound. However, the mean squared error weighs all of the errors equally. This leads to the potential case that the model is initially incorrect, but later along the trajectory it corrects itself. The model hence learns a fundamentally wrong trajectory, and cannot easily be trained out of this error. This can be seen in the following training attempt:

In [12]:
loss(x, y) = sum(abs2, x - y)

l = mean(valid) do v
    loss( model(v), v[2] )
end

θ = 1f-4
η = 1f-3
opt = Flux.OptimiserChain(Flux.WeightDecay(θ), Flux.RMSProp(η))
opt_state = Flux.setup(opt, model) 

N_epochs = 30
for i_e = 1:N_epochs

    Flux.train!(model, train, opt_state) do m, t, x
        result = m((t,x))
        loss(result, x)
    end 

    global l = mean(valid) do v
        loss( model(v), v[2] )
    end
    
    if i_e % 30 == 0
        η /= 2
        Flux.adjust!(opt_state, η)
    end

end

l

0.8005889f0

In [13]:
model(valid[1])

3×8 Matrix{Float32}:
  1.96761    1.95582    1.95118   …   1.97309    1.9906     2.01263
 -0.419014  -0.362848  -0.308322     -0.155707  -0.108696  -0.0636159
  3.42939    3.57166    3.69606       3.9637     4.01901    4.05842

In [14]:
valid[1][2]

3×8 Matrix{Float32}:
  1.96761    1.96363    1.97027   …   2.05794    2.11109    2.17688
 -0.419014  -0.360524  -0.301429     -0.121323  -0.0606422  0.000227584
  3.42939    3.55798    3.66715       3.8768     3.90691    3.91691

In [15]:
fig, ax = plot_nde(sol, model, train, ndata=150)
fig

This error, and the following solution, was discovered in the development of this project. For a dynamical system, we expect the error to compound exponentially. Hence it would seem beneficial to ensure that the model stays as close to the true solution at the _beginning_ of the trajectory as possible. To encourage this, we add an _exponential weight factor_:
$$
L(\mathbf{x}, \mathbf{\hat{x}}) = \sum_{i=1}^d \beta^i \cdot (\mathbf{x}_i - \mathbf{\hat{x}}_i) ^2
$$
where $\beta \in (0,1)$. While optimizing parameters, $\beta$ can be optimized as well. During testing, an optimal value of $\beta = 0.99$ was observed. 

In [16]:
nn = Chain(
    Dense(3 => N_weights),

    SkipConnection(Dense(N_weights => N_weights, swish), +),
    SkipConnection(Dense(N_weights => N_weights, swish), +),
    SkipConnection(Dense(N_weights => N_weights, swish), +),

    Dense(N_weights => 3)
)

p, re_nn = Flux.destructure(nn)
#p = p |> gpu
neural_ode(u, p, t) = re_nn(p)(u)
neural_ode(u) = neural_ode(u, p, 0f0)

neural_ode_prob = ODEProblem(neural_ode, #=CuArray(x0)=#x0, tspan, p)
model = ChaoticNDE(neural_ode_prob, alg=RK4(), gpu=false#=true=#, sensealg=InterpolatingAdjoint())

ChaoticNDE{Vector{Float32}, ODEProblem{SVector{3, Float32}, Tuple{Float32, Float32}, false, Vector{Float32}, ODEFunction{false, SciMLBase.AutoSpecialize, typeof(neural_ode), LinearAlgebra.UniformScaling{Bool}, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, typeof(SciMLBase.DEFAULT_OBSERVED), Nothing, Nothing}, Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}, SciMLBase.StandardODEProblem}, RK4{typeof(OrdinaryDiffEq.trivial_limiter!), typeof(OrdinaryDiffEq.trivial_limiter!), Static.False}, Base.Pairs{Symbol, InterpolatingAdjoint{0, true, Val{:central}, Nothing}, Tuple{Symbol}, NamedTuple{(:sensealg,), Tuple{InterpolatingAdjoint{0, true, Val{:central}, Nothing}}}}, ChaoticNDETools.CPUDevice}(Float32[0.0013786439, 0.36164373, 0.06233575, -0.5600019, 0.0070916433, -0.41426957, -0.046009857, -0.13224953, -0.5695917, -0.14461528  …  -0.27039543, -0.045790855, 0.10235235, -0.07394716, 0.49376255, 0.39751393, 0.

In [17]:
β = 0.99f0
loss(x, y, β) = sum(enumerate(x - y)) do z
    i, xy = z
    β^i * abs2(xy)
end

l = mean(valid) do v
    loss( model(v), v[2], 1f0 )
end

θ = 1f-4
η = 1f-3
opt = Flux.OptimiserChain(Flux.WeightDecay(θ), Flux.RMSProp(η))
opt_state = Flux.setup(opt, model) 

N_epochs = 30
for i_e = 1:N_epochs

    Flux.train!(model, train, opt_state) do m, t, x
        result = m((t,x))
        loss(result, x, β)
    end 

    global l = mean(valid) do v
        loss( model(v), v[2], 1f0 )
    end
    
    if i_e % 30 == 0
        η /= 2
        Flux.adjust!(opt_state, η)
    end

end

l

1.6587579f0

In [18]:
model(valid[1])

3×8 Matrix{Float32}:
  1.96761    1.96927   1.97537   …   2.01491    2.03483    2.05897
 -0.419014  -0.37134  -0.323033     -0.173708  -0.122962  -0.072392
  3.42939    3.55985   3.67484       3.92529    3.97637    4.01073

In [19]:
valid[1][2]

3×8 Matrix{Float32}:
  1.96761    1.96363    1.97027   …   2.05794    2.11109    2.17688
 -0.419014  -0.360524  -0.301429     -0.121323  -0.0606422  0.000227584
  3.42939    3.55798    3.66715       3.8768     3.90691    3.91691

In [23]:
fig, ax = plot_nde(sol, model, train, ndata=150)
fig